In [3]:
import os

In [4]:
os.chdir('../')
%pwd

'/home/eman/Desktop/DataScience_proj'

Things to do:
- feature engineering
- Data preprocessing 

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    unzip_data_dir: Path
    TRAIN_DATA: str
    TEST_DATA: str
    params: dict

In [ ]:
from src.Datascience_proj.utils.common import read_yaml, create_directories
from src.Datascience_proj.constants import *

import pandas as pd
from sklearn.model_selection import train_test_split
from src.Datascience_proj import logger

In [34]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
        ## read the file and assign to variable
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        ##creating our artifacts directory
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
        config= self.config.data_transformation
        params = self.params.data_params

        ## creating a new directory
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            TRAIN_DATA=config.TRAIN_DATA,
            TEST_DATA=config.TEST_DATA,
            params = params
        )

        return data_transformation_config

In [40]:
## creating the Data transformation directory

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config= config

    ## Note: you can add different data transformation techniques such as scaler, PCA, and all
    ##You can perform all kinds of EDA in ML Cycle here before passing data to the model

    # I am only adding train_test_splitting cos this data is already cleaned up

    def train_test_split(self):

        ## importing the dataset as a dataframe
        data = pd.read_csv(self.config.unzip_data_dir)

        ## splitting the dataset using train test split
        train_data, test_data = train_test_split(data, test_size=self.config.params.test_ratio, 
                                                 random_state=self.config.params.random_state)
        
        ## saving the splitted files
        train_data.to_csv(self.config.TRAIN_DATA, index= False)
        test_data.to_csv(self.config.TEST_DATA, index=False)

        logger.info('Splitted data into training and test datasets')
        logger.info(train_data.shape)
        logger.info(test_data.shape)

        print(train_data.shape)
        print(test_data.shape)


In [41]:
config = ConfigurationManager()
data_transform=config.get_data_transformation_config()
transform_data= DataTransformation(data_transform)
transform_data.train_test_split()

[2025-07-16 09:44:09,453: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-16 09:44:09,460: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-16 09:44:09,466: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-16 09:44:09,470: INFO: common: created directory at: artifacts]
[2025-07-16 09:44:09,473: INFO: common: created directory at: artifacts/data_transformation]


[2025-07-16 09:44:09,729: INFO: 2999293573: Splitted data into training and test datasets]
[2025-07-16 09:44:09,732: INFO: 2999293573: (1279, 12)]
[2025-07-16 09:44:09,736: INFO: 2999293573: (320, 12)]
(1279, 12)
(320, 12)


In [37]:
data_transform

DataTransformationConfig(root_dir='artifacts/data_transformation', unzip_data_dir='artifacts/data_ingestion/winequality-red.csv', TRAIN_DATA='artifacts/data_transformation/train.csv', TEST_DATA='artifacts/data_transformation/test.csv', params=ConfigBox({'test_ratio': 0.2, 'random_state': 42}))

In [39]:
data_transform.params.test_ratio

0.2